# RAG com LCEL

Agora vamos para exemplos mais avançados de chains e como podemos reproduzí-las utilizando LCEL.

Vamos começar mostrando como fazer o processos de RAG. Para isso, o processo de criação da vector store é exatamente igual ao que vimos no curso de Aplicações com IA com LangChain.

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

CRIANDO CHUNKS - Carregando e particionando o documento

In [5]:
PATHS = [
    "arquivos\Explorando o Universo das IAs com Hugging Face.pdf"
]

pages = []

for path in PATHS:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

In [12]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap = 50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(pages)

EMBEDDINGS - Importando e instanciando o OpenAiEmbeddings

In [9]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

C:\Users\edinocencio\AppData\Local\Temp\ipykernel_8548\22710313.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


STORAGE - Amrazenando os dados em um VECTOR STORE

In [15]:
from langchain_community.vectorstores.faiss import FAISS

vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,

)

retriever = vector_store.as_retriever(search_type="mmr")


Agora na criação da chain é que temos uma mudança. Podemos definir nossa chain manualmente da seguinte forma

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import  StrOutputParser
from langchain_openai import ChatOpenAI
from regex import template

llm = ChatOpenAI()

prompt_template = """ Reponda as seguintes perguntas ao usuário utilizando apenas o contexto fornecido.

Contexto: {context}

Pergunta: {pergunta}

"""
prompt_template = ChatPromptTemplate.from_template(prompt_template)
output_parser = StrOutputParser()

In [21]:
from langchain_core.runnables import  RunnablePassthrough, RunnableParallel

# RunnableParallel: Provavelmente é uma classe que permite executar múltiplos componentes em paralelo.
# 'pergunta': Um componente que utiliza RunnablePassthrough, que parece ser uma classe que simplesmente passa os dados sem modificações.
# 'context': Um componente que utiliza retriever, que deve ser um objeto ou função responsável por recuperar algum tipo de dados ou contexto.

setup_and_retrievel = RunnableParallel(
    {
        'pergunta': RunnablePassthrough(),
        'context': retriever
     }
)

chain = setup_and_retrievel | prompt_template | llm | output_parser


In [22]:
chain.invoke("O que é Hugging Face?")

'Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'

Entendendo O RunnableParallel e o RunnablePassThrough

In [23]:
{'pergunta': RunnablePassthrough().invoke('O que é LangChain?')}

{'pergunta': 'O que é LangChain?'}

In [26]:
RunnableParallel(
    {
        'pergunta': RunnablePassthrough(),
        'context': retriever
     }
).invoke("O que é Hugging Face?")

{'pergunta': 'O que é Hugging Face?',
 'context': [Document(id='b0af4c33-94b3-41a2-882f-2eb1d53d59c5', metadata={'source': 'arquivos\\Explorando o Universo das IAs com Hugging Face.pdf', 'page': 5, 'page_label': '5'}, page_content='Explorando o Universo das IAs com Hugging Face\n01. O que é Hugging Face?\nBem-vindos ao curso de Hugging Face da Asimov Academy!\nNeste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos\ncomo utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face\naos nossos scripts de Python.\nAfinal, o que é Hugging Face?\nA Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'),
  Document(id='4bdc3bdd-fef9-43c3-a02a-3902b9231c9d', metadata={'source': 'arquivos\\Explorando o Universo das IAs com Hugging Face.pdf', 'page': 88, 'page_label': '88'}, page_content='Face.\nÉ claro que o Hugging Face não acaba aqui. Não tivemos tempo de explorar boa parte das 

# Uma alternativa não paralelizável

Da seguinte forma, també obtemos o mesmo resultado, mas da forma não paralelizável, o que pode gerar um atraso no processamento de nossa chain.

In [28]:
setup_dict = {'pergunta': RunnablePassthrough(), 'context': retriever}
chain = setup_dict | prompt_template | llm | output_parser
chain.invoke("O que é Hugging Face?")

'Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'

# FALLBACKS ## ~Caminho alternativo

Ao trabalhar com modelos de linguagem, você pode encontrar problemas nas APIs subjacentes, seja por limitação de taxa ou tempo de inatividade. Portante, à medida que você move suas aplicações LLM para produção, torna-se cada vez mais importante proteger-se contra esses problemas. É por isso que introduzimos o conceito de fallbacks, ou alternativas em português.

Uma alternativa é um plano substituto que pode ser usado em uma emergência.

Criticamente, as alternativas podem ser aplicadas não apenas no nível do LLM, mas em todo o nível executável. Isso é importante porque, muitas vezes, modelos diferentes exigem prompts diferentes. Então, se sua chamada para a OpenAI falhar, você não quer simplesmente enviar o mesmo prompt para Anthropic - você provavelmente vai querer usar um template de prompt diferente e enviar uma versão diferente lá.


# Fallbacks para entradas grandes

Quando construímos aplicações, precisamos sempre atentar às questões econômicas que envolvem colocar um modelo em produção. Muitas vezes será necessário otimizar custos, evitrando utilizar modelos maiores (mais caros) para problemas simples. Com fallbacks, temos a alternativca de sempre tentar processar e entrada do usuário com um modelo mais complexo.

In [32]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template('Resuma o seguinte texto: {texto}')

chain_pequena = prompt | llm 
chain_pequena.invoke({'texto': 'Oi, eu sou o Edward'})

'\n\n"Eu sou o Edward." '

In [ ]:
chain_pequena.invoke({'texto': 'Oi, eu sou o Edward' * 1000})

In [34]:
from langchain_openai import  ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(model = 'gpt-3.5-turbo-0125')
prompt = ChatPromptTemplate.from_template('Resuma o seguinte texto: {texto}')

chain_grande = prompt | model

In [35]:
chain_grande.invoke({'texto': 'Oi, eu sou o Edward' * 1000})

AIMessage(content='Oi, eu sou o Edward, este é um trecho repetitivo de Edward se apresentando várias vezes.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 6015, 'total_tokens': 6040, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7724a63f-3674-4942-88a5-c876ffcb7b45-0', usage_metadata={'input_tokens': 6015, 'output_tokens': 25, 'total_tokens': 6040, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain_fallback = chain_pequena.with_fallbacks(chain_grande)